In [45]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import glob
import os

# Define the paths to the training and testing datasets
train_path = 'C:/Users/joann/Desktop/haarcascade/train'
test_path = 'C:/Users/joann/Desktop/haarcascade/img'

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

In [47]:
def haar_cascade(image,face_cascade, name, i):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(500, 500))
    return faces

In [48]:
# Set the path of the main folder containing subfolders of images
main_folder = 'C:/Users/joann/Desktop/haarcascade/train'

# Load the Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Set the path of the folder to save the cropped face images
save_folder = glob.glob('C:/Users/joann/Desktop/haarcascade/train/*')


name_dic=[]
for dir in save_folder:
  print(dir)
  dir = dir[40:]
  # print(dir)
  name_dic.append(dir)
  
print(name_dic)
for name in name_dic:
    imgs_dir=glob.glob('C:/Users/joann/Desktop/haarcascade/train/'+ name + '/*' )
    i=0
    for img_dir in imgs_dir:
      
      if img_dir.endswith('.jpg') or img_dir.endswith('.png') :
            image=cv2.imread(img_dir)
            
            faces= haar_cascade(image,face_cascade, name, i)
            
            for (x, y, w, h) in faces:
                    # Draw a rectangle around the detected face
                    rectangle = cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
                    cv2.imwrite('C:/Users/joann/Desktop/haarcascade/squared_test/' + name + '/' + name + str(i) + '.png', rectangle)
                    # Crop the image to only contain the detected face region
                    face = image[y:y+h, x:x+w]
                    # Resize the face to a fixed size for consistency
                    face = cv2.resize(face, (500, 500))
                    # Save the cropped face image to the save folder
                    cv2.imwrite('C:/Users/joann/Desktop/haarcascade/cropped_test/' + name + '/' + name + str(i) + '.png', face)
                    i=i+1


[]


In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.nn.functional as F

In [50]:
class FaceCNN(nn.Module):
    def __init__(self):
        super(FaceCNN, self).__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2, 2)

        # Fully connected layers
        #self.fc1 = nn.Sigmoid()
        self.fc1 = nn.Linear(65536, 2304)
        
        self.fc2 = nn.Linear(2304, 200)
        #self.fc2 = nn.Linear(2304, 128)
        self.relu5 = nn.ReLU()
        #self.fc2 = nn.Linear(128,6)
        
        self.fc3 = nn.Linear(200, 3)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = torch.flatten(x, 1)
        x = F.relu(x)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
       
        x = self.fc3(x)
        x = F.sigmoid(x)
        return x


In [51]:

# Define the data transformers
data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [52]:


# Initialize the model
model = FaceCNN()

# Define the loss function and optimizer 
# The loss function measures the difference between the predicted output of the model and the actual output (the ground truth) for a given input.
#The goal is to minimize this difference, or loss, during training, so that the model learns to make more accurate predictions.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
# Load the dataset
train_dataset = ImageFolder('/content/drive/MyDrive/image_m3/cropped_new/', transform=data_transforms)

# Split the dataset into training and validation sets
# The validation set is used to evaluate the model's performance on unseen data, and prevent overfitting.
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Define the data loaders
batch_size = 25
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)



In [60]:

# Set the device to run the model on (CPU or GPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the number of epochs to train for


# Train the model
# Set the number of epochs to train for
num_epochs = 10
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Define the dataset and dataloader
train_dataset = ImageFolder('/content/drive/MyDrive/image_m3/cropped_new', transform=data_transforms)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=25, shuffle=True)

# test_dataset = ImageFolder('/content/drive/MyDrive/image_m3/test', transform=data_transforms)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=True)


#Train the model for the specified number of epochs
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()

    # Loop through the dataloader and train the model on each batch of images
    for images, labels in train_dataloader:
        # Clear the gradients
        optimizer.zero_grad()
       
       #e: Compute predicted outputs by passing inputs to the model
        outputs = model(images)
        
        # Compute the loss
        new_label = F.one_hot(labels, num_classes=3) 
        loss = criterion(outputs, new_label.type(torch.FloatTensor) )

        # Backward pass: Compute gradient of the loss with respect to model parameters
        loss.backward()

        # Update the model parameters
        optimizer.step()

    # Print the loss every 10 epochs
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10], Loss: 0.6164
Epoch [2/10], Loss: 0.5519
Epoch [3/10], Loss: 0.5841
Epoch [4/10], Loss: 0.6429
Epoch [5/10], Loss: 0.5840
Epoch [6/10], Loss: 0.6426
Epoch [7/10], Loss: 0.7337
Epoch [8/10], Loss: 0.7071
Epoch [9/10], Loss: 0.5838
Epoch [10/10], Loss: 0.7014


In [61]:
# Initialize the running loss and accuracy
running_loss = 0.0
running_corrects = 0

# Loop over the training batches
for inputs, labels in train_loader:
    # Move the inputs and labels to the device
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # Zero the parameter gradients
    optimizer.zero_grad()
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    # Update the running loss and accuracy
    _, preds = torch.max(outputs, 1)
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

# Calculate the average loss and accuracy for the epoch
epoch_loss = running_loss / len(train_dataset)
epoch_acc = running_corrects.double() / len(train_dataset)

# Print the epoch statistics
print(f'Train - Epoch: {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

# Set the model to evaluation mode
model.eval()

# Initialize the running loss and accuracy for the validation set
running_loss = 0.0
running_corrects = 0

# Loop over the validation batches
for inputs, labels in val_loader:
    # Move the inputs and labels to the device
    inputs = inputs.to(device)
    labels = labels.to(device)
    
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    # Update the running loss and accuracy
    _, preds = torch.max(outputs, 1)
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)

# Calculate the average loss and accuracy for the validation set
epoch_loss = running_loss / len(val_dataset)
epoch_acc = running_corrects.double() / len(val_dataset)

# Print the validation statistics
print(f'Validation - Epoch: {epoch+1}/{num_epochs}, Loss: {epoch_loss}, Accuracy: {epoch_acc:.4f}')


Train - Epoch: 10/10, Loss: 0.4963, Accuracy: 0.7143
Validation - Epoch: 10/10, Loss: 0.7233420014381409, Accuracy: 0.8889


# ***after many many trials, we have concluded that batch size = 25, number of epochs = 10, and model learning rate = 0.0001 has produced the highest accuracy in both train and validation***
initial results were: Train - Epoch: 10/10, Loss: 0.9151, Accuracy: 0.6364 
Validation - Epoch: 10/10, Loss: 1.218111515045166, Accuracy: 0.3333 